## Test Co-Design FPS
This file tests the average FPS of the IsaacLab envs. <br>
It tests both the FPS for classic RL training on the tasks as well as the Co-Design FPS,<br>
which may differ due to the reset times between loading unique morphologies.

In [1]:
import time
import subprocess
import os

from stable_baselines3 import PPO

In [2]:
# EA parameters
timesteps_per_generation=13e4
pop_size = 8
generations = 3
ENV_COUNT = 1024

def print_stats(start_time, end_time, timesteps):
    time_diff = end_time - start_time
    print(f"Time taken: {time_diff} seconds, average FPS: {timesteps/(time_diff*ENV_COUNT)}")

### IsaacLab Testing

#### Regular RL FPS

In [ ]:
start_time = time.time()
result = subprocess.run(["python", "sb3/train.py", "--task", "Grasp-Object", 
                         "--headless", "--num_envs", "1024", "--seed", "200",
                         "--max_steps", f"{timesteps_per_generation*generations*pop_size}"], 
                        capture_output=True, text=True)
end_time = time.time()
print(result.stdout)

# Find the latest model path
base_dir = "logs/sb3/Grasp-Object"
model_dir = os.listdir(base_dir)[-1]
model_path = os.path.join(base_dir, model_dir, "model.zip")
model = PPO.load(model_path)

print_stats(start_time, end_time, model.num_timesteps)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'logs/sb3/Grasp-Object'

#### Co-Design FPS

In [ ]:
start_time = time.time()
result = subprocess.run(["python", "population_evolution.py", "--task", "Grasp-Object", 
                         "--headless", "--num_envs", "1024", "--seed", "200", 
                         "--generations", f"{int(generations)}", "--population_size", f"{int(pop_size)}", 
                         "--timesteps_per_agent", f"{int(timesteps_per_generation)}", "--mutation_rate", "0"],
                        capture_output=True, text=True)
end_time = time.time()

# Find the latest model path
base_dir = "logs/sb3/population_evolution/grasp_object"
model_dirs = os.listdir(base_dir)
model_paths = [os.path.join(base_dir, d, f"{d}_gen_10.zip") for d in model_dirs]
models_timesteps = sum([PPO.load(p).num_timesteps for p in model_paths])

print_stats(start_time, end_time)

AttributeError: module 'os' has no attribute 'join'